# 1. Rysowanie Dłonie

Rozpoznawanie dłonie polega na wyznaczeniu pozycji elementów charakterystycznych dłoni. W sumie można wyznaczych ich 21. Są to między innymi stawy, nadgarstek lub końcówki palców. Współrzędne są obliczne względem położenia nagarstka.  

<img src=https://i.imgur.com/qpRACer.png />

Rozpoczynamy od zaimportowania odpowiednich bibliotek.

OpenCV pozowli na przeprowadzenie wstępnych przekształceń obrazu, w taki sposób, aby biblioteka MediaPipe mogła poprawnie rozpoznać dłoń oraz jej elementy charaktterystyczne. 

In [1]:
import mediapipe as mp
import cv2
import numpy as np

Wybieramy dwa obiekty klasy mp.solutions:

1. mp_drawing - pozowli na naniesienie punktów na elementy charakterystyczne dłoni oraz linii ich łączących. 
2. mp_hands - zostanie wykorzystany do rozpoznania dłoni z wybraną dokładnością. 

In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

Wstępne ropoznanie dłoni i nanisieni grafiki na obraz pobrany z kamery.

# 2. Zapis pozycji elementów charakterystycznych do pliku CSV

In [3]:
import csv
import os
import numpy as np

Tworzymy oznaczenia kolumn (klasy, współrzędne)

In [4]:
landmarks = ['class']
for val in range(0, 21):
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val)]

In [ ]:
landmarks

Tworzymy plik CSV i zapisujemy do niego oznaczenia kolumn.

In [5]:
FILE_NAME='wrist_normalized_positions.csv'

In [6]:
with open(FILE_NAME, mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

Tworzymy zmienną class_name, która będzie przechowywała informację o aktualnie przechwytywanym geście. W momencie rozpoczęcie tej części programu, będziemy zapisywać wszystkie współrzędne elementów charakterystycznych dla wybranego gestu. 

In [11]:
class_name="fist"

In [12]:
cap = cv2.VideoCapture(0)

detections = 0
with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5) as hands:
    while cap.isOpened():
        ret, frame = cap.read()
        
        #BGR to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        #Flip horizontal
        image = cv2.flip(image, 1)
        
        #Set flag
        image.flags.writeable = False
        
        #Detections
        results = hands.process(image)
        
        #Set flag back to True
        image.flags.writeable = True
        
        #RGB to BGR
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        #print(results)
        
        #Rendering results
        if results.multi_hand_landmarks:
            for num, hand in enumerate(results.multi_hand_landmarks):
                mp_drawing.draw_landmarks(image, hand, mp_hands.HAND_CONNECTIONS, 
                                         mp_drawing.DrawingSpec(color=(0,255,0), thickness=2, circle_radius=4), 
                                         mp_drawing.DrawingSpec(color=(0,0,255), thickness=2, circle_radius=4))
                
        try:
            hand_landmarks = results.multi_hand_landmarks[0].landmark
            wrist = hand_landmarks[0]
                
            hand_landmarks_row = np.zeros((20,3))
            for i in range(1, len(hand_landmarks)):
                hand_landmarks_row[i-1]=[hand_landmarks[i].x-wrist.x, hand_landmarks[i].y-wrist.y, hand_landmarks[i].z-wrist.z]
                
            print(hand_landmarks_row)
            hand_landmarks_row = hand_landmarks_row.flatten()
            hand_landmarks_row = list(hand_landmarks_row/np.max(np.absolute(hand_landmarks_row)))
            
            hand_landmarks_row.insert(0, class_name)
            
            
            with open(FILE_NAME, mode='a', newline='') as f:
                csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(hand_landmarks_row)
                detections += 1
        except:
            pass
        
        if detections == 500:
            break

        #image = cv2.flip(image, 0)
        cv2.imshow("Gesture Training", image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

[[-0.05577219 -0.02565324 -0.00853794]
 [-0.09684789 -0.07562715 -0.02015979]
 [-0.11394864 -0.11815226 -0.03598246]
 [-0.11738813 -0.14580429 -0.0475488 ]
 [-0.08697325 -0.16379124 -0.0059037 ]
 [-0.10903603 -0.1747722  -0.03051111]
 [-0.09431398 -0.12682915 -0.03802633]
 [-0.07853764 -0.1131376  -0.04013007]
 [-0.05383897 -0.16910613 -0.01790482]
 [-0.07285738 -0.1625089  -0.04400858]
 [-0.06237894 -0.11199069 -0.04327595]
 [-0.05286485 -0.11204475 -0.03829878]
 [-0.01993227 -0.16335964 -0.03426231]
 [-0.03683156 -0.15124154 -0.05744832]
 [-0.0318799  -0.1041075  -0.05021019]
 [-0.02343911 -0.1019212  -0.04085012]
 [ 0.01407897 -0.15158701 -0.05295084]
 [-0.00343657 -0.13768017 -0.06242749]
 [-0.00302583 -0.10497546 -0.05664223]
 [ 0.00431275 -0.10192305 -0.04988037]]
[[-0.05566245 -0.02451682 -0.00427693]
 [-0.10001224 -0.07018828 -0.01657196]
 [-0.11385959 -0.110147   -0.03148563]
 [-0.10502678 -0.13400906 -0.04236563]
 [-0.07449824 -0.16965455 -0.00809092]
 [-0.08461386 -0.1274809

[[-0.02993077  0.00590193 -0.02041062]
 [-0.05433428 -0.00639671 -0.04047208]
 [-0.05131972 -0.02426493 -0.05866922]
 [-0.02666777 -0.04057056 -0.07316991]
 [-0.04999125 -0.10742366 -0.03475285]
 [-0.01218307 -0.05500352 -0.05624361]
 [-0.01352918 -0.02472639 -0.06801619]
 [-0.02561587 -0.02761501 -0.07655956]
 [-0.02234858 -0.1305694  -0.0389931 ]
 [ 0.01717436 -0.05764431 -0.06569974]
 [ 0.00879437 -0.02414757 -0.07027552]
 [-0.00578558 -0.03062671 -0.07391042]
 [ 0.00498545 -0.12968177 -0.04406304]
 [ 0.03847629 -0.05828285 -0.05926742]
 [ 0.02833611 -0.0296436  -0.05279864]
 [ 0.01461798 -0.03826034 -0.04928857]
 [ 0.03148967 -0.11745918 -0.05236932]
 [ 0.05186915 -0.0663026  -0.05496521]
 [ 0.04530555 -0.04027653 -0.05037685]
 [ 0.03549629 -0.04401094 -0.04860356]]
[[-0.02626157  0.00502533 -0.02732569]
 [-0.05115432 -0.00987613 -0.04911393]
 [-0.04820585 -0.03730375 -0.06688526]
 [-0.02635539 -0.06030834 -0.08048137]
 [-0.05881125 -0.10792917 -0.04235544]
 [-0.0132153  -0.0679455

[[-0.06231427 -0.03669673  0.00481227]
 [-0.10509741 -0.08429384 -0.00652191]
 [-0.1285643  -0.11912745 -0.02053439]
 [-0.13693166 -0.14731801 -0.03014007]
 [-0.07735032 -0.1787712  -0.01315796]
 [-0.13474029 -0.12024498 -0.03381646]
 [-0.11549389 -0.08874094 -0.04151982]
 [-0.09530735 -0.09374452 -0.04575693]
 [-0.053527   -0.16079116 -0.03154918]
 [-0.11303431 -0.08815014 -0.0533923 ]
 [-0.09393114 -0.06691968 -0.05172818]
 [-0.07578254 -0.07749367 -0.05164459]
 [-0.03416288 -0.12756556 -0.05057573]
 [-0.08993071 -0.05768663 -0.06569975]
 [-0.07313216 -0.04293561 -0.05448262]
 [-0.05654657 -0.05675763 -0.04787767]
 [-0.01950473 -0.08563733 -0.07002162]
 [-0.06153238 -0.03705293 -0.07633521]
 [-0.05218035 -0.02778208 -0.06836913]
 [-0.03854585 -0.04124171 -0.06350613]]
[[-0.06358594 -0.03605801  0.00357214]
 [-0.11062485 -0.07969677 -0.01098025]
 [-0.14015657 -0.11135536 -0.02793622]
 [-0.15546751 -0.13454628 -0.04078028]
 [-0.07982868 -0.1641264  -0.02340666]
 [-0.13301075 -0.1101788

[[ 6.07657433e-03  4.61897850e-02 -3.12787816e-02]
 [ 2.72780657e-03  8.17816257e-02 -6.29805624e-02]
 [-1.12593174e-04  1.06335163e-01 -9.02792141e-02]
 [-7.09444284e-03  1.28931046e-01 -1.16139874e-01]
 [-4.97148037e-02  4.05285954e-02 -7.26412162e-02]
 [ 1.22531056e-02  4.67650294e-02 -1.01441704e-01]
 [ 1.09875798e-02  5.85991740e-02 -1.16472445e-01]
 [-5.96404076e-03  6.17281795e-02 -1.26065299e-01]
 [-5.01066446e-02 -7.63827562e-03 -7.30561465e-02]
 [ 2.88436413e-02  2.40814686e-03 -9.87475514e-02]
 [ 2.08913088e-02  9.92840528e-03 -1.00509048e-01]
 [-2.32547522e-03  8.90755653e-03 -1.05326101e-01]
 [-3.86128426e-02 -4.30719256e-02 -7.29473829e-02]
 [ 3.49911451e-02 -2.93838978e-02 -8.84069651e-02]
 [ 2.75794268e-02 -2.31911540e-02 -8.25636238e-02]
 [ 7.98171759e-03 -2.56631970e-02 -8.26361850e-02]
 [-2.07477808e-02 -7.09404349e-02 -7.62139782e-02]
 [ 3.19506526e-02 -5.92702627e-02 -8.32162425e-02]
 [ 3.01157832e-02 -5.03098965e-02 -8.05937573e-02]
 [ 1.67885423e-02 -4.97052670e-

[[-0.0290457  -0.05704117 -0.01016907]
 [-0.04420936 -0.11673999 -0.03014145]
 [-0.05103654 -0.15428257 -0.04985453]
 [-0.05499697 -0.17562777 -0.06741751]
 [ 0.01284444 -0.15523112 -0.03746109]
 [-0.05959463 -0.14066839 -0.06225649]
 [-0.05572456 -0.11421418 -0.07559689]
 [-0.03565472 -0.10649008 -0.08414986]
 [ 0.02572823 -0.12016749 -0.04934729]
 [-0.05609173 -0.09293228 -0.0744886 ]
 [-0.04513824 -0.0740239  -0.0775999 ]
 [-0.02488172 -0.07198364 -0.08175525]
 [ 0.0279296  -0.07632315 -0.06120837]
 [-0.05103552 -0.05398679 -0.077776  ]
 [-0.03509521 -0.04132175 -0.07034185]
 [-0.01339281 -0.04287535 -0.06735747]
 [ 0.02220178 -0.03211588 -0.0747297 ]
 [-0.03393853 -0.02039158 -0.08204366]
 [-0.0253877  -0.01441294 -0.0766522 ]
 [-0.00944507 -0.0178712  -0.07453042]]
[[-0.03802204 -0.07881159  0.00874447]
 [-0.04384059 -0.14950484 -0.00077161]
 [-0.04457694 -0.19487888 -0.01440693]
 [-0.04905146 -0.22854954 -0.02357501]
 [ 0.02946347 -0.2078855  -0.00657634]
 [-0.03838354 -0.212345 

...

[[-0.06896341 -0.08155835 -0.01150247]
 [-0.11533797 -0.16898692 -0.03845076]
 [-0.12953764 -0.21909004 -0.06599267]
 [-0.10467756 -0.21889645 -0.08655348]
 [-0.00636542 -0.29951602 -0.03463738]
 [-0.0981406  -0.25328025 -0.07022698]
 [-0.09035826 -0.18329895 -0.08854423]
 [-0.06467879 -0.15994436 -0.1000035 ]
 [ 0.0333699  -0.27311471 -0.05315235]
 [-0.06511182 -0.20921308 -0.09292539]
 [-0.05622548 -0.14714938 -0.09780691]
 [-0.03285187 -0.13557714 -0.10165051]
 [ 0.0585041  -0.22146708 -0.07411125]
 [-0.03639543 -0.16398448 -0.1006527 ]
 [-0.0334425  -0.11700475 -0.09146626]
 [-0.01433957 -0.11025989 -0.08495833]
 [ 0.07025516 -0.15749919 -0.09744968]
 [ 0.00099808 -0.12300885 -0.10610174]
 [-0.00634557 -0.09164172 -0.09818853]
 [ 0.00695753 -0.08588028 -0.0934057 ]]
[[-0.07073712 -0.09368593  0.00845877]
 [-0.10757285 -0.1747368  -0.00694165]
 [-0.12377131 -0.22760987 -0.02631441]
 [-0.11274111 -0.2545042  -0.0399128 ]
 [ 0.01849508 -0.29006884 -0.01633959]
 [-0.07393014 -0.3096994

[[-0.04199028 -0.04867733 -0.02099112]
 [-0.06073427 -0.12457854 -0.03706597]
 [-0.02874225 -0.1711244  -0.05405122]
 [ 0.02066731 -0.15915638 -0.06567451]
 [-0.00706339 -0.22382325 -0.01153615]
 [-0.00907022 -0.24466133 -0.0409021 ]
 [-0.02472383 -0.18671024 -0.05060636]
 [-0.02231461 -0.16229469 -0.05370628]
 [ 0.03951687 -0.20691416 -0.01906115]
 [ 0.03524297 -0.21317148 -0.04892866]
 [ 0.00696874 -0.1472016  -0.04860836]
 [ 0.00360596 -0.13820076 -0.04312101]
 [ 0.07490546 -0.17514831 -0.0331663 ]
 [ 0.06341088 -0.1644997  -0.05994135]
 [ 0.03211695 -0.11141819 -0.05464949]
 [ 0.02394342 -0.10963565 -0.04547569]
 [ 0.1020031  -0.1339336  -0.05102246]
 [ 0.08043975 -0.11489463 -0.06191368]
 [ 0.0520094  -0.08292717 -0.05868563]
 [ 0.04200548 -0.08597928 -0.05311436]]
[[-0.03297389 -0.05170047 -0.03469144]
 [-0.03416181 -0.11681294 -0.05897507]
 [ 0.00359535 -0.14243841 -0.08026414]
 [ 0.04931593 -0.13215858 -0.0962145 ]
 [ 0.0298171  -0.21333075 -0.03851002]
 [ 0.01115221 -0.2099418

[[-0.03099728  0.01707023 -0.03892921]
 [-0.0766058  -0.00249654 -0.06114838]
 [-0.08891487 -0.04167378 -0.07932238]
 [-0.06513262 -0.07477295 -0.0933196 ]
 [-0.10364735 -0.08463985 -0.03890584]
 [-0.08280015 -0.11601925 -0.06979792]
 [-0.06559092 -0.07784873 -0.08245166]
 [-0.06617457 -0.05475509 -0.08824185]
 [-0.07253385 -0.10927153 -0.03339665]
 [-0.05019456 -0.12959546 -0.07031099]
 [-0.0310539  -0.0722546  -0.07334872]
 [-0.03681195 -0.05518156 -0.06879364]
 [-0.04182673 -0.12189358 -0.03349381]
 [-0.02099347 -0.13434207 -0.06847077]
 [-0.00946152 -0.0811035  -0.06275658]
 [-0.01790476 -0.06236917 -0.05142733]
 [-0.01018596 -0.130602   -0.03774526]
 [ 0.00340229 -0.13916624 -0.05884129]
 [ 0.0114795  -0.09689814 -0.05595271]
 [ 0.00265902 -0.07996798 -0.04825218]]
[[-0.03375977  0.0217973  -0.03590872]
 [-0.08980817 -0.01111627 -0.0480058 ]
 [-0.10493147 -0.05860531 -0.05505957]
 [-0.08220458 -0.08471268 -0.05829507]
 [-0.0875811  -0.07668263 -0.0293692 ]
 [-0.08747518 -0.1236276

[[-0.04211068 -0.05662185  0.00846098]
 [-0.06627983 -0.11285824  0.00196575]
 [-0.07610565 -0.1529507  -0.00983209]
 [-0.06848848 -0.17671096 -0.01748572]
 [-0.00278407 -0.19090778  0.00879861]
 [-0.05001026 -0.20230901 -0.00776716]
 [-0.05612671 -0.15627235 -0.01392211]
 [-0.04508919 -0.13852566 -0.01673806]
 [ 0.0143708  -0.18640703 -0.00931595]
 [-0.04145032 -0.18972075 -0.02774304]
 [-0.04368359 -0.13735145 -0.02692467]
 [-0.0283519  -0.12498593 -0.02440112]
 [ 0.02714801 -0.17191935 -0.02976735]
 [-0.02744126 -0.17128599 -0.04350953]
 [-0.02990299 -0.12443578 -0.03547862]
 [-0.01481777 -0.11349851 -0.02870269]
 [ 0.03569072 -0.15276515 -0.05085194]
 [-0.00994772 -0.15126759 -0.05343689]
 [-0.0127328  -0.11902064 -0.0459265 ]
 [ 0.00094217 -0.10999328 -0.04031759]]
[[-0.0437575  -0.04589736 -0.00097485]
 [-0.07568866 -0.09791219 -0.00531815]
 [-0.08289558 -0.14321554 -0.01479064]
 [-0.06513411 -0.16929352 -0.02000569]
 [-0.04431134 -0.15176886  0.016747  ]
 [-0.06329572 -0.1870383

[[-0.0337435  -0.008443   -0.02573542]
 [-0.07009375 -0.05700195 -0.0419272 ]
 [-0.07137835 -0.09748822 -0.05569193]
 [-0.03758323 -0.10169905 -0.0648388 ]
 [-0.05412382 -0.14918816 -0.01976515]
 [-0.03906941 -0.14395261 -0.03875659]
 [-0.03684777 -0.09686363 -0.04621512]
 [-0.03834265 -0.09059709 -0.05063717]
 [-0.01988536 -0.15485555 -0.01700658]
 [-0.0101887  -0.14227545 -0.04303143]
 [-0.01375574 -0.08496791 -0.0451753 ]
 [-0.01934302 -0.08452982 -0.0422931 ]
 [ 0.00790077 -0.14611137 -0.0179049 ]
 [ 0.01653373 -0.13117617 -0.03897401]
 [ 0.00961369 -0.07961565 -0.0331913 ]
 [ 0.00096178 -0.07695508 -0.02461457]
 [ 0.03398108 -0.13269395 -0.02188268]
 [ 0.03838396 -0.12135541 -0.02994546]
 [ 0.03153747 -0.08508241 -0.02545467]
 [ 0.02187103 -0.08474553 -0.01933122]]
[[-0.03609645 -0.0076896  -0.0201729 ]
 [-0.07618886 -0.05253887 -0.03556079]
 [-0.0846495  -0.09261489 -0.04901681]
 [-0.05393785 -0.09859174 -0.05760752]
 [-0.06401581 -0.14433199 -0.01603989]
 [-0.05293572 -0.1403869

In [13]:
len(hand_landmarks_row)

61

# 3. Trening modeli z wykorzystaniem Scikit Learn

In [32]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [33]:
df = pd.read_csv(FILE_NAME)

In [34]:
df.head()

class        x1        y1        z1        x2        y2        z2        x3  \
0  open -0.146620 -0.045271 -0.056340 -0.268748 -0.167505 -0.103869 -0.372308   
1  open -0.135531 -0.085285 -0.034892 -0.261595 -0.215405 -0.068388 -0.372492   
2  open -0.137282 -0.095127 -0.029132 -0.263381 -0.228613 -0.059946 -0.371076   
3  open -0.136066 -0.094340 -0.028677 -0.261508 -0.227710 -0.059921 -0.369200   
4  open -0.134751 -0.095885 -0.030786 -0.257945 -0.232768 -0.061459 -0.364362   

         y3        z3  ...       z17       x18       y18       z18       x19  \
0 -0.260336 -0.155067  ... -0.180461  0.218943 -0.521408 -0.246330  0.271010   
1 -0.312089 -0.108833  ... -0.138293  0.199882 -0.556778 -0.181943  0.249662   
2 -0.324944 -0.099272  ... -0.146282  0.198206 -0.555813 -0.191231  0.251571   
3 -0.323561 -0.099914  ... -0.151023  0.198049 -0.549592 -0.197083  0.250386   
4 -0.334263 -0.099845  ... -0.145053  0.201637 -0.550712 -0.189891  0.254488   

        y19       z19       x20       y20       z20  
0 -0.623706 -0.280494  0.314938 -0.718661 -0.304452  
1 -0.650513 -0.205114  0.290371 -0.738695 -0.222150  
2 -0.646993 -0.212016  0.296578 -0.734171 -0.226651  
3 -0.640731 -0.217898  0.295313 -0.727351 -0.232431  
4 -0.641479 -0.211782  0.299414 -0.728569 -0.227409  

[5 rows x 61 columns]

In [35]:
df.tail()

class        x1        y1        z1        x2        y2        z2  \
995  fist -0.234491 -0.079259 -0.115513 -0.376922 -0.264912 -0.213577   
996  fist -0.216485 -0.087769 -0.135969 -0.411460 -0.335732 -0.228750   
997  fist -0.217903 -0.054522 -0.166190 -0.452640 -0.368098 -0.270750   
998  fist -0.236879 -0.050462 -0.132382 -0.499981 -0.344781 -0.233364   
999  fist -0.255509 -0.103082 -0.106428 -0.503423 -0.390117 -0.206435   

           x3        y3        z3  ...       z17       x18       y18  \
995 -0.301143 -0.429766 -0.302442  ... -0.281867  0.336219 -0.597053   
996 -0.387929 -0.563911 -0.309410  ... -0.210107  0.292517 -0.710378   
997 -0.460935 -0.629543 -0.359638  ... -0.141310  0.247869 -0.783668   
998 -0.555503 -0.607775 -0.321668  ... -0.130203  0.184618 -0.772014   
999 -0.568429 -0.643673 -0.302989  ... -0.180508  0.153220 -0.715471   

          z18       x19       y19       z19       x20       y20       z20  
995 -0.297580  0.277608 -0.426440 -0.275486  0.202985 -0.448760 -0.259903  
996 -0.259494  0.232541 -0.494987 -0.236362  0.161725 -0.476506 -0.203471  
997 -0.193377  0.203657 -0.549431 -0.164377  0.141235 -0.547255 -0.124834  
998 -0.145150  0.157026 -0.532588 -0.105522  0.085863 -0.552676 -0.067734  
999 -0.207089  0.100254 -0.484783 -0.157221  0.068748 -0.503093 -0.105165  

[5 rows x 61 columns]

In [36]:
x = df.drop('class', axis=1)
y = df['class']

In [37]:
x

x1        y1        z1        x2        y2        z2        x3  \
0   -0.146620 -0.045271 -0.056340 -0.268748 -0.167505 -0.103869 -0.372308   
1   -0.135531 -0.085285 -0.034892 -0.261595 -0.215405 -0.068388 -0.372492   
2   -0.137282 -0.095127 -0.029132 -0.263381 -0.228613 -0.059946 -0.371076   
3   -0.136066 -0.094340 -0.028677 -0.261508 -0.227710 -0.059921 -0.369200   
4   -0.134751 -0.095885 -0.030786 -0.257945 -0.232768 -0.061459 -0.364362   
..        ...       ...       ...       ...       ...       ...       ...   
995 -0.234491 -0.079259 -0.115513 -0.376922 -0.264912 -0.213577 -0.301143   
996 -0.216485 -0.087769 -0.135969 -0.411460 -0.335732 -0.228750 -0.387929   
997 -0.217903 -0.054522 -0.166190 -0.452640 -0.368098 -0.270750 -0.460935   
998 -0.236879 -0.050462 -0.132382 -0.499981 -0.344781 -0.233364 -0.555503   
999 -0.255509 -0.103082 -0.106428 -0.503423 -0.390117 -0.206435 -0.568429   

           y3        z3        x4  ...       z17       x18       y18  \
0   -0.260336 -0.155067 -0.473759  ... -0.180461  0.218943 -0.521408   
1   -0.312089 -0.108833 -0.467977  ... -0.138293  0.199882 -0.556778   
2   -0.324944 -0.099272 -0.462799  ... -0.146282  0.198206 -0.555813   
3   -0.323561 -0.099914 -0.458938  ... -0.151023  0.198049 -0.549592   
4   -0.334263 -0.099845 -0.455990  ... -0.145053  0.201637 -0.550712   
..        ...       ...       ...  ...       ...       ...       ...   
995 -0.429766 -0.302442 -0.110373  ... -0.281867  0.336219 -0.597053   
996 -0.563911 -0.309410 -0.194392  ... -0.210107  0.292517 -0.710378   
997 -0.629543 -0.359638 -0.242699  ... -0.141310  0.247869 -0.783668   
998 -0.607775 -0.321668 -0.353961  ... -0.130203  0.184618 -0.772014   
999 -0.643673 -0.302989 -0.389560  ... -0.180508  0.153220 -0.715471   

          z18       x19       y19       z19       x20       y20       z20  
0   -0.246330  0.271010 -0.623706 -0.280494  0.314938 -0.718661 -0.304452  
1   -0.181943  0.249662 -0.650513 -0.205114  0.290371 -0.738695 -0.222150  
2   -0.191231  0.251571 -0.646993 -0.212016  0.296578 -0.734171 -0.226651  
3   -0.197083  0.250386 -0.640731 -0.217898  0.295313 -0.727351 -0.232431  
4   -0.189891  0.254488 -0.641479 -0.211782  0.299414 -0.728569 -0.227409  
..        ...       ...       ...       ...       ...       ...       ...  
995 -0.297580  0.277608 -0.426440 -0.275486  0.202985 -0.448760 -0.259903  
996 -0.259494  0.232541 -0.494987 -0.236362  0.161725 -0.476506 -0.203471  
997 -0.193377  0.203657 -0.549431 -0.164377  0.141235 -0.547255 -0.124834  
998 -0.145150  0.157026 -0.532588 -0.105522  0.085863 -0.552676 -0.067734  
999 -0.207089  0.100254 -0.484783 -0.157221  0.068748 -0.503093 -0.105165  

[1000 rows x 60 columns]

In [38]:
y

0      open
1      open
2      open
3      open
4      open
       ... 
995    fist
996    fist
997    fist
998    fist
999    fist
Name: class, Length: 1000, dtype: object

Wszystkie pobrane dane dzielimy na dwie części, pierwsza posłuży do trenowania, druga do testwowania.

In [39]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=3451)

In [40]:
type(y_train)

pandas.core.series.Series

In [60]:
x_train

x1        y1        z1        x2        y2        z2        x3  \
295 -0.035629  0.232968 -0.043324 -0.011994  0.477322 -0.071968 -0.009959   
296 -0.039112  0.261612 -0.059029  0.002728  0.505605 -0.093814  0.036809   
117 -0.115756 -0.094467 -0.045335 -0.205137 -0.248485 -0.076004 -0.252581   
422 -0.120190 -0.146129 -0.033639 -0.214612 -0.332566 -0.062690 -0.298423   
569 -0.251047 -0.056709 -0.177873 -0.471921 -0.499975 -0.245164 -0.487758   
..        ...       ...       ...       ...       ...       ...       ...   
456 -0.124067 -0.127122 -0.049448 -0.205519 -0.322295 -0.081350 -0.258275   
97  -0.100588 -0.181948  0.043062 -0.111176 -0.344241  0.038390 -0.115052   
150 -0.273113 -0.139721 -0.015699 -0.515996 -0.256601 -0.110291 -0.696028   
654 -0.209562 -0.217053 -0.107099 -0.370875 -0.585262 -0.142848 -0.356548   
95  -0.085520 -0.129094 -0.006123 -0.141853 -0.279898 -0.030523 -0.195159   

           y3        z3        x4  ...       z17       x18       y18  \
295  0.680060 -0.104086 -0.043056  ... -0.134080  0.502493  0.151886   
296  0.722041 -0.123951  0.025599  ... -0.102165  0.548744  0.071545   
117 -0.395605 -0.110644 -0.306168  ... -0.129821  0.202047 -0.536166   
422 -0.474778 -0.099622 -0.384221  ... -0.115528  0.343621 -0.499584   
569 -0.892079 -0.298002 -0.331607  ... -0.056195  0.217406 -0.781516   
..        ...       ...       ...  ...       ...       ...       ...   
456 -0.492177 -0.116457 -0.318430  ... -0.104785  0.323107 -0.489022   
97  -0.453424  0.016356 -0.127505  ... -0.171235  0.031842 -0.485539   
150 -0.313608 -0.221384 -0.848862  ... -0.464632 -0.131077 -0.086332   
654 -0.888083 -0.173143 -0.224973  ... -0.057194  0.230851 -0.736305   
95  -0.390037 -0.066706 -0.250327  ... -0.155452  0.166919 -0.497106   

          z18       x19       y19       z19       x20       y20       z20  
295 -0.198900  0.586816  0.143101 -0.233015  0.663804  0.145985 -0.253204  
296 -0.166480  0.636702  0.041115 -0.204000  0.721652  0.017621 -0.226739  
117 -0.177253  0.249665 -0.621616 -0.205952  0.286595 -0.704603 -0.226921  
422 -0.156135  0.421529 -0.566086 -0.177845  0.489295 -0.632292 -0.192739  
569 -0.139350  0.186405 -0.550814 -0.116283  0.168511 -0.451530 -0.063015  
..        ...       ...       ...       ...       ...       ...       ...  
456 -0.149721  0.395204 -0.561779 -0.178853  0.459973 -0.631361 -0.198954  
97  -0.203442  0.036560 -0.590756 -0.212362  0.046125 -0.678377 -0.220576  
150 -0.568714 -0.191851  0.185762 -0.558069 -0.249233  0.401536 -0.547018  
654 -0.105412  0.197599 -0.552162 -0.083788  0.146929 -0.472555 -0.045601  
95  -0.194781  0.201792 -0.583881 -0.212735  0.234184 -0.671769 -0.227650  

[700 rows x 60 columns]

# 4. Trenowanie Klasyfikujących Modeli Uczenia Maszynowego

In [61]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

Tworzymy słownik przechowywujący 4 metody uczenie maszynowego wraz z metodą normalizacji.

In [62]:
pipelines = {
    'lr':make_pipeline(StandardScaler(), LogisticRegression()),
    'rd':make_pipeline(StandardScaler(), RidgeClassifier()),
    'rf':make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gb':make_pipeline(StandardScaler(), GradientBoostingClassifier()),
}

In [25]:
y_train

295    open
296    open
117    open
422    open
569    fist
       ... 
456    open
97     open
150    open
654    fist
95     open
Name: class, Length: 700, dtype: object

Trenujemy 4 różne modele jednocześnie. 

!!! PRZETESTOWAĆ INNE METODY !!!

In [63]:
fit_models = {}

for algo, pipeline in pipelines.items():
    model = pipeline.fit(x_train, y_train)
    fit_models[algo] = model

In [64]:
fit_models['rf'].predict(x_test)

array(['open', 'open', 'open', 'fist', 'open', 'open', 'open', 'fist',
       'open', 'fist', 'open', 'open', 'fist', 'open', 'fist', 'fist',
       'fist', 'fist', 'open', 'fist', 'open', 'fist', 'open', 'fist',
       'open', 'fist', 'fist', 'fist', 'fist', 'fist', 'open', 'fist',
       'fist', 'open', 'fist', 'open', 'fist', 'open', 'fist', 'open',
       'open', 'open', 'fist', 'fist', 'fist', 'fist', 'open', 'fist',
       'fist', 'fist', 'open', 'open', 'open', 'fist', 'fist', 'open',
       'fist', 'fist', 'open', 'open', 'fist', 'fist', 'open', 'fist',
       'fist', 'fist', 'open', 'open', 'fist', 'open', 'fist', 'open',
       'fist', 'open', 'open', 'open', 'open', 'open', 'open', 'open',
       'open', 'fist', 'open', 'fist', 'open', 'fist', 'fist', 'fist',
       'fist', 'open', 'fist', 'open', 'open', 'open', 'open', 'fist',
       'open', 'open', 'open', 'open', 'open', 'open', 'open', 'open',
       'open', 'fist', 'open', 'open', 'open', 'open', 'open', 'fist',
      

# 5. Ewaluacja Modelu

In [65]:
from sklearn.metrics import accuracy_score
import pickle

Porównujemy dokładność każdego modelu wykorzystując funkcję accuracy_score

In [66]:
for algo, model in fit_models.items():
    yhat = model.predict(x_test)
    print(algo, accuracy_score(y_test, yhat))

lr 0.9966666666666667
rd 0.9933333333333333
rf 0.9866666666666667
gb 0.9933333333333333


In [67]:
with open('gesture_recognition.pkl', 'wb') as f:
    pickle.dump(fit_models['lr'], f)

# 5. Detekcje

Powtórnie ładujemy model.

In [68]:
with open('gesture_recognition.pkl', 'rb') as f:
    model = pickle.load(f)

In [49]:
model

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('logisticregression', LogisticRegression())])

In [50]:
model.predict(x_test)

array(['open', 'open', 'open', 'fist', 'open', 'open', 'open', 'fist',
       'open', 'fist', 'open', 'open', 'fist', 'open', 'fist', 'fist',
       'fist', 'fist', 'open', 'fist', 'open', 'fist', 'open', 'fist',
       'open', 'fist', 'fist', 'fist', 'fist', 'fist', 'open', 'fist',
       'fist', 'open', 'fist', 'open', 'fist', 'open', 'fist', 'open',
       'open', 'fist', 'fist', 'fist', 'fist', 'fist', 'open', 'fist',
       'fist', 'fist', 'open', 'open', 'open', 'fist', 'fist', 'open',
       'fist', 'fist', 'open', 'open', 'fist', 'fist', 'open', 'fist',
       'fist', 'fist', 'open', 'open', 'fist', 'open', 'fist', 'open',
       'fist', 'open', 'open', 'open', 'open', 'open', 'open', 'open',
       'open', 'fist', 'open', 'fist', 'open', 'fist', 'fist', 'fist',
       'fist', 'open', 'fist', 'open', 'open', 'open', 'open', 'fist',
       'open', 'open', 'open', 'open', 'open', 'open', 'open', 'open',
       'open', 'fist', 'open', 'open', 'open', 'open', 'open', 'fist',
      

In [51]:
pd.DataFrame(x_test)

x1        y1        z1        x2        y2        z2        x3  \
311 -0.122366  0.154571 -0.031917 -0.199848  0.348932 -0.042279 -0.261205   
472 -0.120036 -0.118734 -0.029220 -0.216168 -0.285095 -0.058979 -0.286563   
387 -0.018149 -0.325881 -0.009420  0.062272 -0.614014 -0.007430  0.150967   
560 -0.289820  0.108541 -0.205375 -0.734906 -0.080346 -0.264466 -0.946463   
423 -0.117827 -0.138752 -0.033318 -0.211735 -0.323980 -0.066010 -0.290154   
..        ...       ...       ...       ...       ...       ...       ...   
766  0.106971 -0.360454  0.092596  0.081788 -0.660642  0.103109 -0.037583   
181  0.210523  0.083584 -0.035197  0.428733  0.052988 -0.064152  0.603579   
896 -0.284715 -0.232632  0.005019 -0.449822 -0.521568 -0.017280 -0.466985   
637 -0.300201  0.012477 -0.129525 -0.658218 -0.200261 -0.237174 -0.849936   
765  0.032872 -0.372588  0.102737 -0.045548 -0.657220  0.114579 -0.167819   

           y3        z3        x4  ...       z17       x18       y18  \
311  0.517757 -0.050960 -0.327744  ... -0.003933  0.232572  0.484272   
472 -0.416182 -0.097176 -0.357530  ... -0.132790  0.215612 -0.546907   
387 -0.824606 -0.017904  0.192865  ... -0.034878  0.672843 -0.164905   
560 -0.400162 -0.294812 -0.861793  ...  0.026708 -0.162364 -1.000000   
423 -0.462984 -0.106554 -0.373721  ... -0.126488  0.336963 -0.495040   
..        ...       ...       ...  ...       ...       ...       ...   
766 -0.805570  0.092885 -0.151875  ... -0.126584 -0.328395 -0.709786   
181  0.009858 -0.095681  0.747452  ... -0.142963  0.289274 -0.735975   
896 -0.781971 -0.079262 -0.379222  ... -0.252696  0.082159 -0.875625   
637 -0.453128 -0.338899 -0.803452  ... -0.243938 -0.012868 -0.759016   
765 -0.788998  0.099934 -0.262368  ... -0.145308 -0.441985 -0.628679   

          z18       x19       y19       z19       x20       y20       z20  
311 -0.016964  0.259329  0.536242 -0.026011  0.283341  0.584262 -0.032395  
472 -0.171076  0.242087 -0.638599 -0.188892  0.254576 -0.727021 -0.201592  
387 -0.062415  0.775462 -0.187457 -0.075748  0.870415 -0.223513 -0.079016  
560 -0.044242 -0.080222 -0.733392 -0.014111 -0.096771 -0.623113  0.046288  
423 -0.167951  0.411554 -0.564842 -0.191383  0.474222 -0.633846 -0.208356  
..        ...       ...       ...       ...       ...       ...       ...  
766 -0.071641 -0.302990 -0.609321 -0.037710 -0.262762 -0.541080 -0.025406  
181 -0.197838  0.332259 -0.869786 -0.231096  0.372779 -0.984128 -0.254844  
896 -0.313326  0.076459 -0.670760 -0.265768  0.146113 -0.561699 -0.207085  
637 -0.292509  0.006543 -0.533807 -0.265404 -0.017603 -0.560098 -0.232358  
765 -0.095732 -0.405659 -0.526059 -0.059475 -0.349275 -0.466140 -0.044703  

[300 rows x 60 columns]

In [56]:
cap = cv2.VideoCapture(0)

detections = 0
with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5) as hands:
    while cap.isOpened():
        ret, frame = cap.read()
        
        #BGR to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        #Flip horizontal
        image = cv2.flip(image, 1)
        
        #Set flag
        image.flags.writeable = False
        
        #Detections
        results = hands.process(image)
        
        #Set flag back to True
        image.flags.writeable = True
        
        #RGB to BGR
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        #Rendering results
        if results.multi_hand_landmarks:
            for num, hand in enumerate(results.multi_hand_landmarks):
                mp_drawing.draw_landmarks(image, hand, mp_hands.HAND_CONNECTIONS, 
                                         mp_drawing.DrawingSpec(color=(0,255,0), thickness=2, circle_radius=4), 
                                         mp_drawing.DrawingSpec(color=(0,0,255), thickness=2, circle_radius=4))
                
        try:
            hand_landmarks = results.multi_hand_landmarks[0].landmark
            wrist = hand_landmarks[0]
                
            hand_landmarks_row = np.zeros((20,3))
            for i in range(1, len(hand_landmarks)):
                hand_landmarks_row[i-1]=[hand_landmarks[i].x-wrist.x, hand_landmarks[i].y-wrist.y, hand_landmarks[i].z-wrist.z]
                
            print(hand_landmarks_row)
            hand_landmarks_row = hand_landmarks_row.flatten()
            hand_landmarks_row = list(hand_landmarks_row/np.max(np.absolute(hand_landmarks_row)))
            
            #Make Detections
            x = pd.DataFrame([hand_landmarks_row])

            #gesture_prob = model.predict_proba(x)[0]

            with warnings.catch_warnings():
               warnings.filterwarnings("ignore")
            print(model.predict(x))
            cv2.putText(image, gesture_class, (10,20), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)

        except:
            pass
        

        #image = cv2.flip(image, 0)
        cv2.imshow("Hand Tracking", image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    print(image.shape)

    cap.release()
    cv2.destroyAllWindows()

[[-0.07018876 -0.02732176 -0.02297506]
 [-0.12993568 -0.09078765 -0.04257974]
 [-0.17974818 -0.13912779 -0.06454919]
 [-0.22859877 -0.16444272 -0.08780117]
 [-0.07431167 -0.23884666 -0.02245026]
 [-0.10547215 -0.34116131 -0.04814701]
 [-0.12302411 -0.40541899 -0.06730437]
 [-0.13587147 -0.46089947 -0.08234037]
 [-0.02243459 -0.25189823 -0.0370304 ]
 [-0.02970964 -0.36726958 -0.05966332]
 [-0.03762114 -0.44024086 -0.07681262]
 [-0.04292274 -0.499816   -0.0920242 ]
 [ 0.02457446 -0.2323001  -0.05638395]
 [ 0.04082733 -0.33893305 -0.08888503]
 [ 0.04792923 -0.40707976 -0.11019827]
 [ 0.05321223 -0.46310142 -0.12407006]
 [ 0.0675953  -0.18605077 -0.0784105 ]
 [ 0.10550994 -0.24469152 -0.10810884]
 [ 0.13130373 -0.2761305  -0.12215807]
 [ 0.15172964 -0.30276936 -0.13143843]]
[[-0.07076108 -0.0388366  -0.01774829]
 [-0.12913185 -0.1046809  -0.03646932]
 [-0.17811531 -0.15338796 -0.05909754]
 [-0.2284134  -0.17937517 -0.08325849]
 [-0.07335633 -0.25550011 -0.01596064]
 [-0.10165173 -0.3598305

[[-0.0817976  -0.01358706 -0.0183415 ]
 [-0.15595603 -0.10249525 -0.02515795]
 [-0.19052976 -0.19289041 -0.03360875]
 [-0.2010805  -0.24936908 -0.03558646]
 [-0.13200849 -0.18519789  0.0013221 ]
 [-0.15433389 -0.2384426  -0.02012976]
 [-0.13181424 -0.1608876  -0.01742617]
 [-0.11876363 -0.14898449 -0.01253858]
 [-0.08324552 -0.20396748 -0.00104475]
 [-0.10702795 -0.23868501 -0.03063731]
 [-0.08944643 -0.14232838 -0.01957053]
 [-0.0820595  -0.14857841 -0.00248154]
 [-0.03752452 -0.20579249 -0.00979761]
 [-0.05649775 -0.2250717  -0.03953443]
 [-0.04805982 -0.13334489 -0.02254859]
 [-0.03917086 -0.13960338 -0.00074691]
 [ 0.00739199 -0.20755935 -0.02147497]
 [-0.01061666 -0.21439758 -0.03712288]
 [-0.01052231 -0.14952308 -0.02790842]
 [-0.00318635 -0.15110642 -0.0126132 ]]
[[-0.07710928  0.00066102 -0.03117448]
 [-0.15500808 -0.09508294 -0.0400498 ]
 [-0.18817019 -0.1941191  -0.04649848]
 [-0.19171721 -0.2477341  -0.04660384]
 [-0.12894815 -0.18183571 -0.0122767 ]
 [-0.15067691 -0.2337589

In [ ]:
hand_landmarks_row